In [22]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [23]:
!pip install opendatasets
!pip install pandas

Библиотеки

In [24]:
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

Скачиваем датасет

In [25]:
od.download("https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression")

Skipping, found downloaded files in "./house-price-prediction-simplified-for-regression" (use force=True to force download)


Чтение датасета

In [26]:
pd.set_option("display.max_columns", None)
df = pd.read_csv("house-price-prediction-simplified-for-regression/Real_Estate.csv")

Приводим таргет к числовому типу, удаляя строки, которые не удалось конвертировать  
В House price of unit area есть цена 0, что может помешать модели, поэтому удаляем  

In [27]:
df["House price of unit area"] = pd.to_numeric(df["House price of unit area"], errors="coerce")
df = df.dropna(subset=["House price of unit area"])

df = df[df["House price of unit area"] > 0].copy()

Формируем признаки и целевую переменную
Таргет - цена
Признаки использую все, кроме цены и времени, осталяя только числовые характеристики  
Делюсь на обучающую и тестовую выборки. В бейзлайне использую линейную регрессию из sklearn с предварительным стандартизированием всех признаков. Считаю три метрики: MAE, RMSE и R^2, чтобы оценить базовое качество линейной модели на исходных признаках

MAE ≈ 9.65 показывает, что средняя ошибка прогноза по цене довольно велика относительно типичных значений. RMSE ≈ 11.37 указывает на наличие отдельных более крупных промахов. R^2 ≈ 0.28 означает, что модель объясняет только ~28% разброса цен

In [28]:
y = df["House price of unit area"]
X = df.drop(columns=["House price of unit area", "Transaction date"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

num_cols = X.columns.tolist()

preprocessor_reg = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
    ]
)

lin_reg_baseline = Pipeline(steps=[
    ("preprocess", preprocessor_reg),
    ("model", LinearRegression())
])

lin_reg_baseline.fit(X_train, y_train)
y_pred_base = lin_reg_baseline.predict(X_test)

mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2_base = r2_score(y_test, y_pred_base)

print("Бейзлайн")
print("MAE:", mae_base)
print("RMSE:", rmse_base)
print("R^2:", r2_base)

Бейзлайн
MAE: 9.652318332589692
RMSE: 11.372057025055133
R^2: 0.28498603386590127


Первая гипотеза - улучшим качество за счет подбора гиперпараметров соседей и типа весов. Используем GridSearchCV с минимизацией MAE

Подбираю коэффициент регуляризации α для Ridge-регрессии по MAE на кросс-валидации. Лучшим оказалось значение 10. На тесте модель даёт MAE ≈ 9.58, RMSE ≈ 11.32 и R^2 ≈ 0.29. По сравнению с бейзлайном линейной регрессии качество немного улучшилось. Ошибка слегка снизилась, а доля объяснённой дисперсии выросла. Регуляризация L2 помогает чуть стабилизировать модель, но прирост качества остаётся умеренным

In [29]:
ridge_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_reg),
    ("model", Ridge(random_state=42))
])

param_grid_ridge = {
    "model__alpha": [0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0]
}

ridge_gs = GridSearchCV(
    estimator=ridge_pipe,
    param_grid=param_grid_ridge,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

ridge_gs.fit(X_train, y_train)

print("Лучшие параметры:", ridge_gs.best_params_)
print("Лучший MAE на CV:", -ridge_gs.best_score_)

best_ridge = ridge_gs.best_estimator_
y_pred_ridge = best_ridge.predict(X_test)

mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
r2_ridge = r2_score(y_test, y_pred_ridge)

print("\nRidge-регрессия")
print("MAE:", mae_ridge)
print("RMSE:", rmse_ridge)
print("R^2:", r2_ridge)

Лучшие параметры: {'model__alpha': 10.0}
Лучший MAE на CV: 9.397653665287969

Ridge-регрессия
MAE: 9.57618614916091
RMSE: 11.321007730022512
R^2: 0.2913910373260141


Гипотеза 2 - взять вместо расстояния до метро его логарифм, так как в распределении данные неравномерные

На кросс-валидации лучшим оказалось очень маленькое значение регуляризации, но на тестовой выборке качество заметно ухудшилось: MAE вырос до ≈10.46, RMSE до ≈12.19, а R² упал до ≈0.18. Это говорит о том, что логарифмирование расстояния до MRT в такой постановке задачи не помогает модели, а, скорее всего, приводит к переобучению под валидационные разбиения и ухудшению обобщающей способности

In [30]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

preprocessor_log = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_log),
    ]
)

ridge_log_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_log),
    ("model", Ridge(random_state=42))
])

param_grid_ridge_log = {
    "model__alpha": [0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0]
}

ridge_log_gs = GridSearchCV(
    estimator=ridge_log_pipe,
    param_grid=param_grid_ridge_log,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

ridge_log_gs.fit(X_train_log, y_train_log)

print("Лучшие параметры:", ridge_log_gs.best_params_)
print("Лучший MAE на CV:", -ridge_log_gs.best_score_)

best_ridge_log = ridge_log_gs.best_estimator_
y_pred_log = best_ridge_log.predict(X_test_log)

mae_log = mean_absolute_error(y_test_log, y_pred_log)
rmse_log = np.sqrt(mean_squared_error(y_test_log, y_pred_log))
r2_log = r2_score(y_test_log, y_pred_log)

print("\nRidge-регрессия")
print("MAE:", mae_log)
print("RMSE:", rmse_log)
print("R^2:", r2_log)


Лучшие параметры: {'model__alpha': 0.01}
Лучший MAE на CV: 9.671738022270157

Ridge-регрессия
MAE: 10.461407475833552
RMSE: 12.193956351577775
R^2: 0.17789793424285816


3 гипотеза - логарифмирование таргета может помочь уменьшить влияние дорогих объектов  
MAE ≈ 10.17, RMSE ≈ 11.70 и R^2 ≈ 0.24 - все три показателя хуже, чем у базового линейного бейзлайна и особенно хуже, чем у Ridge без логарифмирования. То есть для этого датасета логарифмирование таргета в линейной модели скорее портит качество, чем улучшает его, и более разумным выбором остаётся обычная Ridge-регрессия без трансформации цены

In [31]:
y_log_target = np.log1p(df["House price of unit area"])
X_t = df.drop(columns=["House price of unit area", "Transaction date"])

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_t, y_log_target,
    test_size=0.2,
    random_state=42
)

num_cols_t = X_t.columns.tolist()

preprocessor_t = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols_t),
    ]
)

ridge_log_target_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_t),
    ("model", Ridge(random_state=42))
])

param_grid_ridge_t = {
    "model__alpha": [0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0]
}

ridge_log_target_gs = GridSearchCV(
    estimator=ridge_log_target_pipe,
    param_grid=param_grid_ridge_t,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

ridge_log_target_gs.fit(X_train_t, y_train_t)

print("Лучшие параметры:", ridge_log_target_gs.best_params_)
print("Лучший MAE на CV:", -ridge_log_target_gs.best_score_)

best_ridge_t = ridge_log_target_gs.best_estimator_

y_pred_log_t = best_ridge_t.predict(X_test_t)
y_pred_t = np.expm1(y_pred_log_t)

y_test_true = np.expm1(y_test_t)

mae_log_t = mean_absolute_error(y_test_true, y_pred_t)
rmse_log_t = np.sqrt(mean_squared_error(y_test_true, y_pred_t))
r2_log_t = r2_score(y_test_true, y_pred_t)

print("\nRidge-регрессия с логарифмом таргета")
print("MAE:", mae_log_t)
print("RMSE:", rmse_log_t)
print("R^2:", r2_log_t)


Лучшие параметры: {'model__alpha': 10.0}
Лучший MAE на CV: 0.3477069719356893

Ridge-регрессия с логарифмом таргета
MAE: 10.17089597304163
RMSE: 11.698286412394637
R^2: 0.2433745112489608


Реализую собственную линейную регрессию  
Модель поддерживает опцию свободного члена, L2-регуляризацию через параметры penalty="l2" и alpha, а также обучение градиентным спуском. В fit инициализирую веса, считаю предсказания, градиенты MSE-функции потерь (с добавлением L2-штрафа) и обновляю параметры. В predict возвращаю линейную комбинацию признаков и обученных коэффициентов

In [32]:
class MyLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        lr=0.01,
        n_iter=1000,
        fit_intercept=True,
        penalty="none",  # 'none' или 'l2'
        alpha=0.0,
        random_state=None
    ):
        self.lr = lr
        self.n_iter = n_iter
        self.fit_intercept = fit_intercept
        self.penalty = penalty
        self.alpha = alpha
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        n_samples, n_features = X.shape

        rng = np.random.RandomState(self.random_state)
        self.w_ = rng.normal(scale=0.01, size=n_features)
        self.b_ = 0.0 if self.fit_intercept else 0.0

        if self.penalty == "l2":
            lam = float(self.alpha)
        elif self.penalty in ["none", None]:
            lam = 0.0
        else:
            raise ValueError("поддерживаются только penalty='l2' или 'none'")

        for _ in range(self.n_iter):
            y_pred = X @ self.w_ + (self.b_ if self.fit_intercept else 0.0)
            error = y_pred - y

            grad_w = X.T @ error / n_samples + lam * self.w_
            if self.fit_intercept:
                grad_b = error.mean()
            else:
                grad_b = 0.0

            self.w_ -= self.lr * grad_w
            if self.fit_intercept:
                self.b_ -= self.lr * grad_b

        return self

    def predict(self, X):
        X = np.asarray(X)
        return X @ self.w_ + (self.b_ if self.fit_intercept else 0.0)

Строю бейзлайн на собственной линейной регрессии, использую те же признаки, разбиение train/test и стандартизацию, что и для библиотечной модели. В пайплайне вместо модели из sklearn подключаю MyLinearRegression без регуляризации

Ошибки и коэффициент детерминации практически совпадают с результатами библиотечной модели, поэтому реализацию можно считать корректной: модель в среднем ошибается примерно на 9–10 единиц цены за квадратный метр и объясняет около 28 % вариации стоимости, то есть даёт базовое, но далёкое от идеального качество

In [33]:
y = df["House price of unit area"]
X = df.drop(columns=["House price of unit area", "Transaction date"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

num_cols = X.columns.tolist()

preprocessor_reg = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
    ]
)

my_lin_base = Pipeline(steps=[
    ("preprocess", preprocessor_reg),
    ("model", MyLinearRegression(
        lr=0.01,
        n_iter=3000,
        fit_intercept=True,
        penalty="none",
        alpha=0.0,
        random_state=42
    ))
])

my_lin_base.fit(X_train, y_train)
y_pred_my_base = my_lin_base.predict(X_test)

mae_my_base = mean_absolute_error(y_test, y_pred_my_base)
rmse_my_base = np.sqrt(mean_squared_error(y_test, y_pred_my_base))
r2_my_base = r2_score(y_test, y_pred_my_base)

print("Бейзлайн MyLinearRegression")
print("MAE:", mae_my_base)
print("RMSE:", rmse_my_base)
print("R^2:", r2_my_base)

Бейзлайн MyLinearRegression
MAE: 9.65231833255961
RMSE: 11.372057025050223
R^2: 0.28498603386651855


Гипотеза 1

Качество почти не изменилось относительно бейзлайна: MAE снизился с 9.65 до 9.63, RMSE немного уменьшился, а R^2 вырос с 0.285 до 0.287. То есть слабая L2-регуляризация даёт лишь лёгкое сглаживание весов и совсем небольшое улучшение, что в целом совпадает с поведением библиотечной Ridge-регрессии

In [34]:
my_ridge_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_reg),
    ("model", MyLinearRegression(
        lr=0.01,
        n_iter=3000,
        fit_intercept=True,
        penalty="l2",
        alpha=1.0,
        random_state=42
    ))
])

param_grid_my1 = {
    "model__alpha": [0.0, 0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0],
    "model__penalty": ["l2"]
}

my_ridge_gs = GridSearchCV(
    estimator=my_ridge_pipe,
    param_grid=param_grid_my1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_ridge_gs.fit(X_train, y_train)

print("Лучшие параметры", my_ridge_gs.best_params_)
print("Лучший MAE на CV:", -my_ridge_gs.best_score_)

best_my_ridge = my_ridge_gs.best_estimator_
y_pred_my_ridge = best_my_ridge.predict(X_test)

mae_my_ridge = mean_absolute_error(y_test, y_pred_my_ridge)
rmse_my_ridge = np.sqrt(mean_squared_error(y_test, y_pred_my_ridge))
r2_my_ridge = r2_score(y_test, y_pred_my_ridge)

print("\nГипотеза 1")
print("MAE:", mae_my_ridge)
print("RMSE:", rmse_my_ridge)
print("R^2:", r2_my_ridge)


Лучшие параметры {'model__alpha': 0.01, 'model__penalty': 'l2'}
Лучший MAE на CV: 9.406585001772603

Гипотеза 1
MAE: 9.62842947066716
RMSE: 11.354835524871307
R^2: 0.28714998528470204


Гипотеза 2

Итоговое качество на тесте заметно ухудшилось. MAE вырос примерно до 10.46, RMSE до 12.19, а R^2 упал до 0.18 по сравнению с бейзлайном. То есть для линейной модели такая логарифмическая трансформация признака скорее ломает исходную линейную связь с ценой, чем помогает

In [35]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

preprocessor_log = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_log),
    ]
)

my_ridge_log_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_log),
    ("model", MyLinearRegression(
        lr=0.01,
        n_iter=3000,
        fit_intercept=True,
        penalty="l2",
        alpha=1.0,
        random_state=42
    ))
])

param_grid_my2 = {
    "model__alpha": [0.0, 0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0],
    "model__penalty": ["l2"]
}

my_ridge_log_gs = GridSearchCV(
    estimator=my_ridge_log_pipe,
    param_grid=param_grid_my2,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_ridge_log_gs.fit(X_train_log, y_train_log)

print("Лучшие параметры:", my_ridge_log_gs.best_params_)
print("Лучший MAE на CV", -my_ridge_log_gs.best_score_)

best_my_ridge_log = my_ridge_log_gs.best_estimator_
y_pred_my_log = best_my_ridge_log.predict(X_test_log)

mae_my_log = mean_absolute_error(y_test_log, y_pred_my_log)
rmse_my_log = np.sqrt(mean_squared_error(y_test_log, y_pred_my_log))
r2_my_log = r2_score(y_test_log, y_pred_my_log)

print("\nГипотеза 2")
print("MAE:", mae_my_log)
print("RMSE:", rmse_my_log)
print("R^2:", r2_my_log)

Лучшие параметры: {'model__alpha': 0.0, 'model__penalty': 'l2'}
Лучший MAE на CV 9.671727293936891

Гипотеза 2
MAE: 10.461453823842039
RMSE: 12.194004511546039
R^2: 0.177891440454224


Гипотеза 3

Подбор регуляризации (L2, α = 0.1) дал на кросс-валидации очень низкий MAE в лог-масштабе, но на реальных ценах улучшения не получилось. MAE даже немного вырос (≈9.89 против ≈9.65 у бейзлайна), а RMSE и R^2 изменились совсем незначительно (RMSE ≈11.35, R² ≈0.287)

In [36]:
y_log_target = np.log1p(df["House price of unit area"])
X_t = df.drop(columns=["House price of unit area", "Transaction date"])

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_t, y_log_target,
    test_size=0.2,
    random_state=42
)

num_cols_t = X_t.columns.tolist()

preprocessor_t = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols_t),
    ]
)

my_ridge_log_target_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_t),
    ("model", MyLinearRegression(
        lr=0.01,
        n_iter=3000,
        fit_intercept=True,
        penalty="l2",
        alpha=1.0,
        random_state=42
    ))
])

param_grid_my3 = {
    "model__alpha": [0.0, 0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0],
    "model__penalty": ["l2"]
}

my_ridge_log_target_gs = GridSearchCV(
    estimator=my_ridge_log_target_pipe,
    param_grid=param_grid_my3,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_ridge_log_target_gs.fit(X_train_t, y_train_t)

print("Лучшие параметры:", my_ridge_log_target_gs.best_params_)
print("Лучший MAE на CV:", -my_ridge_log_target_gs.best_score_)

best_my_ridge_t = my_ridge_log_target_gs.best_estimator_

y_pred_log_t = best_my_ridge_t.predict(X_test_t)
y_pred_t = np.expm1(y_pred_log_t)

y_test_true = np.expm1(y_test_t)

mae_my_log_t = mean_absolute_error(y_test_true, y_pred_t)
rmse_my_log_t = np.sqrt(mean_squared_error(y_test_true, y_pred_t))
r2_my_log_t = r2_score(y_test_true, y_pred_t)

print("\nГипотеза 3")
print("MAE:", mae_my_log_t)
print("RMSE:", rmse_my_log_t)
print("R^2:", r2_my_log_t)


Лучшие параметры: {'model__alpha': 0.1, 'model__penalty': 'l2'}
Лучший MAE на CV: 0.34755402523057105

Гипотеза 3
MAE: 9.885303321187472
RMSE: 11.353969887126523
R^2: 0.2872586696168604


Простой бейзлайн даёт адекватное качество с MAE около 9.6–9.7 и R^2 примерно 0.28. Добавление L2-регуляризации и подбор коэффициента α через кросс-валидацию позволяют чуть стабилизировать модель и совсем немного улучшить метрики, но без резкого скачка качества. Логарифмирование расстояния до MRT, как и раньше, только ухудшает результат, а логарифмирование таргета в линейной регрессии уже не даёт такого выигрыша, как в KNN: на MAE и RMSE эффект почти нейтральный. Собственная реализация линейной регрессии по качеству практически совпадает с библиотечной Ridge-моделью, так что её можно считать корректной